In [ ]:
from vantage6.client import UserClient as Client

server_url = "https://plugin.dhd.nl"  # e.g. https://cotopaxi.vantage6.ai or
# http://localhost for a local dev server
server_port = 443  # This is specified when you first created the server
server_api = "/api"  # This is specified when you first created the server

username = "dhd-admin"
with open("../.secrets/v6_dhd_key") as f:
    password = f.read().strip()

organization_key = "/home/vantage-node/.local/share/vantage6/node/privkey_DHD.pem"  # This can be empty if you do not want to set up encryption

# Initialize the client object, and run the authentication
client = Client(server_url, server_port, server_api, log_level="debug")
client.authenticate(username, password)

# Optional: setup the encryption, if you have an organization_key
client.setup_encryption(organization_key)

In [ ]:
# creata a task
input_ = {
    "method": "http_folders_test",
    "kwargs": {"http_folder": "http://apache-server:8081"},
}

average_task = client.task.create(
    collaboration=1,
    organizations=[3],
    name="http_folders_test",
    image="drplugindhdprd.azurecr.io/v6-http-tag:latest",
    description="",
    input_=input_,
)

In [ ]:
# wait for the results and print logs
print("Waiting for results")
task_id = average_task["id"]
result = client.wait_for_results(task_id)
print(result)
run = client.run.from_task(average_task["id"])
print(run["data"][0]["log"])

In [ ]:
import logging

# Set up logging for the entire script to error level
logging.basicConfig(level=logging.ERROR)

# Get specific loggers and set them to ERROR level to suppress less severe messages
logging.getLogger("httpcore").setLevel(logging.ERROR)
import logging
import os

from webdav4.fsspec import WebdavFileSystem

# Set up logging
logging.basicConfig(level=logging.ERROR)

http_folder = "http://localhost:8081"


def get_wd_fs(url, credentials: tuple[str, str] | None = None):
    """Get a WebdavFileSystem object."""
    return WebdavFileSystem(base_url=url, auth=credentials)


def listdir(path: str, fs: WebdavFileSystem, detail: bool | None = False):
    """List the files in the given path."""
    files = fs.ls(path, detail=detail)
    return files


def read_file(path: str, fs: WebdavFileSystem) -> str:
    """Read the content of the file in the given path."""
    with fs.open(path, "r") as f:
        return f.read()


def remove_file(path: str, fs: WebdavFileSystem) -> bool:
    """Remove the file in the given path."""
    print(fs.ls(os.path.dirname(path), detail=False))
    if path.lstrip("/") in fs.ls(os.path.dirname(path), detail=False):
        fs.rm(path)
        return path.lstrip("/") not in fs.ls(os.path.dirname(path), detail=False)
    print(f"File {path} not found, cannot remove.")
    return False


def write_file(path: str, content: str, fs: WebdavFileSystem) -> bool:
    """Write the content to the file in the given path."""
    with fs.open(path, "w") as f:
        f.write(content)
    return read_file(path, fs) == content


try:
    fs = get_wd_fs(url=http_folder)
    print(listdir("/data", fs))
except Exception as e:
    print("webdav listdir request unsuccesful 😒")
    print(e)

try:
    fs = get_wd_fs(url=http_folder)
    print(read_file("/data/test.txt", fs))
except Exception as e:
    print("webdav read file request unsuccesful 😒")
    print(e)

try:
    fs = get_wd_fs(url=http_folder)
    path = "/data/test_v6.txt"
    write_result = write_file(path, "Hello World!", fs)
    print(write_result)
    if write_result:
        print(remove_file(path, fs))
    else:
        print("File does not exist or could not be created.")
except Exception as e:
    print("Operation unsuccessful 😒")
    print(e)